In [59]:
#all imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import pandas as pd
from tqdm import tqdm

In [60]:
nodes = pd.read_csv("youtube_raw.csv").drop("id.1",axis = 1)

#eliminate recommendations
nodes = nodes.iloc[:,0:9]

In [61]:
#transform every column in list
for i in nodes.columns:
    globals()[i]=nodes[i].values.tolist()
    

In [62]:
#print all variables that are now available as lists
nodes.columns

Index(['id', 'uploader', 'age', 'category', 'length', 'views', 'rate',
       'ratings', 'comments'],
      dtype='object')

# Define and Compute features

In [63]:
#read train set and train graph
train_edges = pd.read_csv("train_set.csv", index_col =0).values.tolist()
G_train = nx.read_edgelist("train.edgelist", delimiter=',')

#read test set and test graph
test_edges = pd.read_csv("test_set.csv", index_col =0).values.tolist()
G_test = nx.read_edgelist("test.edgelist", delimiter=',')

In [64]:
page_rank_train = nx.pagerank(G_train)
hubs_train, _ = nx.hits(G_train)

page_rank_test = nx.pagerank(G_test)
hubs_test, _ = nx.hits(G_test)

In [65]:
def Random_walk_features(source, target, train=True):
    
    if train:
        #Page rank
        page_rank = page_rank_train
        #Hit
        hubs = hubs_train
    else:
        #Page rank
        page_rank = page_rank_test
        #Hit
        hubs = hubs_test
        
    diff_page_rank = page_rank[source] - page_rank[target]
    diff_hubs = hubs[source] - hubs[target]

    return [diff_page_rank,
            diff_hubs]

In [66]:
#compute features for training set
x_train = []
y_train = []
for i in tqdm(train_edges):
    x_train.append(Random_walk_features(i[0], i[1]))
    y_train.append(int(i[2]))

100%|███████████████████████████████| 307274/307274 [00:02<00:00, 149095.05it/s]


In [67]:
x_train = pd.DataFrame(x_train, columns = [
                                            'diff_page_rank',
                                            'diff_hubs'
                                            ])
            
x_train['y'] = y_train

In [68]:
x_train.to_csv('Random_walk_feautures_train.csv', index=False)

In [69]:
#compute features for test set
x_test = []
y_test = []
for i in tqdm(test_edges):
    x_test.append(Random_walk_features(i[0], i[1], train=False))
    y_test.append(int(i[2]))

100%|███████████████████████████████| 170706/170706 [00:00<00:00, 444439.18it/s]


In [70]:
x_test = pd.DataFrame(x_test, columns = [
                                            'diff_page_rank',
                                            'diff_hubs'
                                            ])
            
x_test['y'] = y_test

In [71]:
x_test.to_csv('Random_walk_feautures_test.csv', index=False)